<a href="https://colab.research.google.com/github/Vaanathipoovannan/TNDSI-Generative-AI_timeforecasting-using-RNN/blob/main/Timewforecasting_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

train_dataset=pd.read_csv('/content/Apple_Train.csv')
test_dataset=pd.read_csv('/content/Apple_Test.csv')

train_dataset.head(5)

In [ ]:
test_dataset.head

In [ ]:

plt.plot(train_dataset['Open'])

In [ ]:
import seaborn as sns
sns.pairplot(train_dataset)
plt.plot(test_dataset['Open'])
plt.xlabel('Time')
plt.ylabel('Stock');



In [ ]:

train_data=train_dataset.iloc[:,1:3].values
train_data.shape

In [ ]:

from sklearn.preprocessing import MinMaxScaler
Scaler=MinMaxScaler()
scaled_data=Scaler.fit_transform(train_data)

X_train=[]
y_train=[]

for i in range(80,len(scaled_data)):
    X_train.append(scaled_data[i-80:i,:])
    y_train.append(scaled_data[i,0])

X_train=np.array(X_train)
y_train=np.array(y_train)
print(type(X_train))
print(type(y_train))

In [ ]:

X_train.shape

In [ ]:

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

model=Sequential()

model.add(LSTM(units=60,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=60,return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1))

model.compile(optimizer='adam',loss='mean_squared_error')

model.fit(X_train,y_train,epochs=100,batch_size=32)

In [ ]:

test_data=test_dataset.iloc[:,1:2].values
final_dataset=pd.concat((train_dataset[['Open','Close']],test_dataset[['Open','Close']]),axis=0)
input_data=final_dataset[len(final_dataset)-len(test_dataset)-80:].values
input_data.reshape(-1,1)

input_data=Scaler.transform(input_data)
X_test=[]
for i in range(80,100):
    X_test.append(input_data[i-80:i,0:2])

X_test=np.array(X_test)

predictions=model.predict(X_test)

trainPredict_extended = np.zeros((len(predictions),2))

trainPredict_extended[:,1] = predictions.transpose()
trainPredict_extended

In [ ]:

trainPredict = Scaler.inverse_transform(trainPredict_extended)[:,1]
plt.plot(trainPredict,label='Predicted')
plt.plot(test_data,label='Actual')
plt.title('Apple Stock Price Pridiction')
plt.xlabel('Time')
plt.ylabel('Stocks')
plt.legend()